# Strands Agent で OpenAI モデルを使用する

## 概要

Strands Agents は、わずか数行のコードで AI エージェントを構築・実行するためのモデル駆動型アプローチを採用した SDK です。Strands は、複数のプロバイダーと、あらゆる場所でホストされているモデルをサポートしています。

[LiteLLM](https://docs.litellm.ai/docs/) は、Amazon、Anthropic、OpenAI など、さまざまな LLM プロバイダーのモデルを単一の API で操作できる統合インターフェースです。Strands Agent SDK は LiteLLM プロバイダーを実装しており、LiteLLM がサポートするあらゆるモデルに対してエージェントを実行できます。

この例では、Microsoft Azure でホストされている `gpt-4.1-mini` モデルを Strands Agent の基盤モデルとして使用する方法を説明します。天気予報と時刻取得ツールを使ったシンプルなエージェントユースケースを使用します。

## エージェントの詳細
<div style="float: left; margin-right: 20px;">

|機能 |説明 |
|--------------------|---------------------------------------------------|
|使用されている機能 |LiteLLM モデル |
|エージェントの構造 |単一エージェントアーキテクチャ |

</div>

## アーキテクチャ

<div style="text-align:center">
<img src="images/architecture.png" width="65%" />
</div>

## 主な機能
* **LiteLLM モデル**: LiteLLM 経由で提供されるモデルを使用

## セットアップと前提条件

### 前提条件
* Python 3.10 以上
* Azure アカウント
* gpt-4.1-mini へのアクセス

Strands エージェントに必要なパッケージをインストールしましょう

In [ ]:
# 前提条件のインストール
!pip install -r requirements.txt

### 依存パッケージのインポート

依存パッケージをインポートしましょう

In [ ]:
import os
from datetime import datetime
from datetime import timezone as tz
from typing import Any
from zoneinfo import ZoneInfo

from strands import Agent, tool
from strands.models.litellm import LiteLLMModel

### Azure キーの設定

Azure API キーを設定しましょう

In [ ]:
os.environ["AZURE_API_KEY"] = "<YOUR_API_KEY>"
os.environ["AZURE_API_BASE"] = "<YOUR_API_BASE>"
os.environ["AZURE_API_VERSION"] = "<YOUR_API_VERSION>"

### カスタムツールの設定

エージェントをテストするためのダミーツールを2つ設定しましょう

In [ ]:
@tool
def current_time(timezone: str = "UTC") -> str:
    if timezone.upper() == "UTC":
        timezone_obj: Any = tz.utc
    else:
        timezone_obj = ZoneInfo(timezone)

    return datetime.now(timezone_obj).isoformat()


@tool
def current_weather(city: str) -> str:
    # ダミー実装です。実際の天気API呼び出しに置き換えてください。
    return "sunny"

### エージェント基盤となるLLMモデルの定義

次に、LiteLLMを使用してエージェント基盤となるモデルを定義します。`gpt-4.1-mini`に設定します。

In [ ]:
model = "azure/gpt-4.1-mini"
litellm_model = LiteLLMModel(
    model_id=model, params={"max_tokens": 32000, "temperature": 0.7}
)

### エージェントの定義

必要な情報がすべて揃ったので、エージェントを定義しましょう。

In [ ]:
system_prompt = "あなたは時間と天気を教えてくれるシンプルなエージェントです"
agent = Agent(
    model=litellm_model,
    system_prompt=system_prompt,
    tools=[current_time, current_weather],
)

### エージェントのテスト

エージェントを呼び出してテストしてみましょう

In [ ]:
results = agent("What time is it in Seattle? And how is the weather?")

#### エージェントの結果の分析

さあ、エージェントを初めて呼び出しました！それでは、結果オブジェクトを見てみましょう。まず、エージェントがエージェントオブジェクト内で交換しているメッセージを確認できます。

In [ ]:
agent.messages

次に、結果のメトリックを分析して、最後のクエリに対するエージェントの使用状況を確認します。

In [ ]:
results.metrics

### おめでとうございます！
このノートブックでは、OpenAI を使用して天気予報エージェントに回答を提供する LiteLLM の使用方法を学びました。